# Laser Modulation

This notebook was used to modulate the lasers. We used a National Instruments PXI-6602 Counter/Timer module for synchronizing the camera with the two lasers.

The first part of the code defines all terminals that we used.
- ctr3 : for the 405 laser
- ctr1 : for the 651 laser
- pfi39 : connected to the camera to synchronize the pulses

Before running the program it is import to specify the exposure time per frame (time per frame), the pulse duration for the 405 and the 561 laser and the time between the two 561 laser pulses (separation_561).

It is also possible to shift all pulses by adjusting the shift variable.

Rijksuniversiteit Groningen, 2021  
Michiel Punter (c.m.punter at rug.nl)  
Dmitrii Linnik (d.linnik at rug.nl)

In [1]:
from PyDAQmx import *
import time

# counters with corresponding terminals
terminal_camera = '/dev1/pfi39'

counter_405 = '/dev1/ctr3'
terminal_405 = '/dev1/pfi24'

counter_561 = '/dev1/ctr1'

counter_sync1 = '/dev1/ctr5'
terminal_sync1 = '/dev1/'

counter_sync2 = '/dev1/ctr6'
terminal_sync2 = '/dev1/'

# exposure time per frame
exposure_time = 0.21

# pulse duration
duration_405 = 500e-6
duration_561 = 500e-6

# shift pulses
shift = 0

# separation between the two 561 pulses
separation_561 = 1e-3

# create tasks

print('create synchronization tasks')

task_sync1 = Task()
task_sync1.CreateCOPulseChanTime(counter_sync1, '', DAQmx_Val_Seconds, DAQmx_Val_High, 0, shift, (exposure_time * 1.5) - shift)
task_sync1.CfgImplicitTiming(DAQmx_Val_FiniteSamps, 1)
task_sync1.SetStartTrigRetriggerable(True)
task_sync1.CfgDigEdgeStartTrig(terminal_camera, DAQmx_Val_Rising)

# make sure the seperation duration is the time between the middle of both pulses
separationduration_561tion_561

# calculate the amount of time we have to wait before we turn on the 561 laser
initial_delay_561 = exposure_time - (shift + duration_561 + (separation_561 / 2.0))

task_sync2 = Task()
task_sync2.CreateCOPulseChanTime(counter_sync2, '', DAQmx_Val_Seconds, DAQmx_Val_High, 0, initial_delay_561, exposure_time * 0.75)
task_sync2.CfgImplicitTiming(DAQmx_Val_FiniteSamps, 1)
task_sync2.SetStartTrigRetriggerable(True)
task_sync2.CfgDigEdgeStartTrig(terminal_camera, DAQmx_Val_Rising)

print('create task for 405')

task_405 = Task()
task_405.CreateCOPulseChanTime(counter_405, '', DAQmx_Val_Seconds, DAQmx_Val_Low, 0, duration_405, duration_405)
task_405.CfgImplicitTiming(DAQmx_Val_FiniteSamps, 1)
task_405.SetStartTrigRetriggerable(True)
task_405.CfgDigEdgeStartTrig(terminal_sync1, DAQmx_Val_Rising)

print('create task for 561')

task_561 = Task()
task_561.CreateCOPulseChanTime(counter_561, '', DAQmx_Val_Seconds, DAQmx_Val_Low, 0, separation_561, duration_561)
task_561.CfgImplicitTiming(DAQmx_Val_FiniteSamps, 2)
task_561.SetStartTrigRetriggerable(True)
task_561.CfgDigEdgeStartTrig(terminal_sync2, DAQmx_Val_Rising)

print('start tasks')

task_405.StartTask()
task_561.StartTask()
task_sync1.StartTask()
task_sync2.StartTask()

input('press enter to stop tasks')

task_405.StopTask()
task_561.StopTask()
task_sync1.StopTask()
task_sync2.StopTask()

time.sleep(2)

print('done')


reset device
create task for 405
create task for 561
start tasks
press enter to stop tasks
done


C:\ProgramData\Anaconda3\lib\site-packages\PyDAQmx\DAQmxFunctions.py:67: StoppedBeforeDoneWarning: Finite acquisition or generation has been stopped before the requested number of samples were acquired or generated.
 in function DAQmxStopTask
  warnings.warn(exception_class(errBuff.value.decode("utf-8"), f.__name__))
